# SC3 - Shape-Controlled Clustering and Christofides algorithm

Final execution test of the chosen algorithm: SC3.

## Installations and imports

In [1]:
!pip install haversine ortools pulp numba

In [2]:
import pandas as pd
from haversine import haversine
from Simulation import Simulation
from GoogleRouting import GoogleRouting
from AdjustedKMeans import AdjustedKMeans

import warnings
warnings.filterwarnings('ignore')

## Data loading and cleaning

In [3]:
FILE_PATH = 'dataset.csv'

Let's remove useless columns and records.

In [4]:
df = pd.read_csv(FILE_PATH)
df['detected_at'] = pd.to_datetime(df['detected_at'])

USELESS_COLUMNS = ['id','raw_data_id','raw_data_setting_id','Seriale','created_at','DataUltimaRilevazione','DataUltimaTrasmissione','DataPrimaInstallazione','Indirizzo','Cap','UnitaTerritoriale','Viario','Tronco', 'Esterno', 'AreaGestionale']
df.drop(USELESS_COLUMNS, axis=1, inplace=True)

df.occluded.replace({1:False, 2:True}, inplace=True)
df.fillna(value=False, inplace=True)
df = df[df.TipoAnomalia == False]
df.drop('TipoAnomalia', axis=1, inplace=True)
df.rename(columns={'Latitudine': 'latitude', 'Longitudine':'longitude'}, inplace=True)

df.set_index('detected_at', inplace=True, drop=True)
df.sort_index(inplace=True)

Remaining columns:

In [5]:
print(df.columns.values[1:])

['bin_serial' 'bin_level' 'occluded' 'latitude' 'longitude']


## Useful functions

Slender distance:

In [6]:
from haversine import haversine
import math

def slender_distance(p1, p2, center, alpha_1=1, alpha_2=0):
    ang_d = math.radians(get_angle(p1, p2, center))
    radial_d = haversine(p1, p2)
    return alpha_1*ang_d+alpha_2*radial_d

def get_angle(a, b, origin):
    ang = math.degrees(math.atan2(b[1]-origin[1], b[0]-origin[0]) - math.atan2(a[1]-origin[1], a[0]-origin[0]))
    ang = abs(ang) if abs(ang) < 180 else 360-abs(ang)
    return ang

# Test configuration

Parameter configuration for the clustering algorithm.

With the parameter *balanced* toggled the number of bins is split equally among clusters.

*distance* is the distance function used in order to create the clusters. We'll use the slender distance with *alpha*=1 e *beta*=0.

In [7]:
clustering_kwargs = {
    'max_size' : 200,
    'balanced': False,
    'distance': lambda p1, p2 : slender_distance(p1, p2, depot, 1, 0),
}

Routing parameters. The number of vehicles is set to one because the algorithm will be run on each cluster separately.

In [8]:
from haversine import haversine

routing_kwargs = {
    'distance_function': haversine,
    'vehicle_capacities': 200,
    'num_vehicles': 1,
}

Final configuration.

In [9]:
SC3_config = {
  'cluster_class': AdjustedKMeans,
  'cluster_kwargs': clustering_kwargs,
  'graph_class': GoogleRouting,
  'graph_kwargs': routing_kwargs,
}

Function to filter the bins that need to be emptied in each time window.

In [10]:
def filter_function(data, level=3):
    new_data = data.drop_duplicates(subset='bin_serial', keep='last')
    new_data = new_data[(new_data.bin_level > level) | new_data.occluded]
    return new_data

## Test

In [11]:
import warnings
warnings.filterwarnings('ignore')

start_date = '2019-09-01 00:00:00'
end_date = '2019-10-01 00:00:00'
data = df[start_date : end_date]

depot = (45.5069182, 9.2684501)
vehicle_capacities = 200
num_vehicles = 20

In [12]:
simulation = Simulation(depot, SC3_config, window_size=6, max_size=200, filter_function=filter_function, filter_kwargs={})
routes = simulation.compute_simulation(data, start_date, end_date, speed=30, emp_time=60, debug=False)
simulation.to_csv('Output\SC3.csv')
score = simulation.get_score()

print('# Output #')
print(f'Numero di turni eseguiti: {str(len(routes))}.')
print(f'Distanza totale: {str(score)} km.')
total_bins = sum([len(routes[w][c]) for w in range(len(routes)) for c in range(len(routes[w]))])
print(f'Numero di cestini svuotati: {str(total_bins)}.')
total_vehs = sum([len(routes[w]) for w in range(len(routes))])
print(f'Numero di veicoli usati: {str(total_vehs)}.')

# Output #
Numero di turni eseguiti: 120.
Distanza totale: 17675.593510000002 km.
Numero di cestini svuotati: 130845.
Numero di veicoli usati: 709.
